In [1]:
!gdown --id 1W9zVRz--bHlbBXbCSmoWHBO_2Cs4EhPY
!unzip vn_stopwords.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1W9zVRz--bHlbBXbCSmoWHBO_2Cs4EhPY
To: /content/vn_stopwords.zip
100% 6.89k/6.89k [00:00<00:00, 9.39MB/s]
Archive:  vn_stopwords.zip
  inflating: vietnamese-stopwords.txt  


In [2]:
!gdown --id 1ydGNBdRVloX9rtxsKrMSnUNFG43Qv1sl
!unzip news_corpus.zip

Streaming output truncated to the last 5000 lines.
 extracting: news_corpus/NEWS_069403.txt  
  inflating: news_corpus/NEWS_001402.txt  
  inflating: news_corpus/NEWS_105164.txt  
  inflating: news_corpus/NEWS_108900.txt  
  inflating: news_corpus/NEWS_095239.txt  
  inflating: news_corpus/NEWS_130689.txt  
  inflating: news_corpus/NEWS_178360.txt  
  inflating: news_corpus/NEWS_042044.txt  
  inflating: news_corpus/NEWS_148066.txt  
  inflating: news_corpus/NEWS_094609.txt  
  inflating: news_corpus/NEWS_151009.txt  
  inflating: news_corpus/NEWS_079889.txt  
  inflating: news_corpus/NEWS_111269.txt  
  inflating: news_corpus/NEWS_128100.txt  
  inflating: news_corpus/NEWS_088832.txt  
  inflating: news_corpus/NEWS_129622.txt  
  inflating: news_corpus/NEWS_029152.txt  
  inflating: news_corpus/NEWS_032313.txt  
  inflating: news_corpus/NEWS_145860.txt  
  inflating: news_corpus/NEWS_042270.txt  
  inflating: news_corpus/NEWS_004225.txt  
  inflating: news_corpus/NEWS_176921.txt  
  i

In [3]:
from nltk.stem.snowball import stopwords
import numpy as np
import nltk
nltk.download('stopwords')
import string
PUNCTUATION = string.punctuation
def LowerCase(text):
  return text.lower()
def RemovePunctuation(text):
    return ''.join([char for char in text if char not in PUNCTUATION])
with open('vietnamese-stopwords.txt', 'r', encoding='utf8') as f:
  vn_stopwords = f.readlines()
def RemoveStopwords(text):
  new_text = text
  for w in vn_stopwords:
    if w in new_text:
      new_text = new_text.replace(w, '')
  return new_text
def Normalize(text):
  normalize_text = text.lower()
  normalize_text = RemovePunctuation(normalize_text)
  normalize_text = RemoveStopwords(normalize_text)
  return normalize_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from tqdm import tqdm
import os

In [23]:
docs_list = list()
vocab = list()

In [30]:
dataset_path = 'news_corpus'
filenames = os.listdir(dataset_path)
for i in tqdm(range(len(filenames)//20)):
  filename = filenames[i]
  file_path = os.path.join(dataset_path,filename)
  with open(file_path, 'r', encoding = 'utf8') as f:
    lines = f.readlines()
    title = lines[0].strip()
    artical = ' '.join(lines[1:]).strip()
    artical = Normalize(artical)
    if (title,artical) not in docs_list:
      docs_list.append((title, artical))
    tokens = artical.split(' ')
    for token in tokens:
      if token not in vocab:
        vocab.append(token)

100%|██████████| 9226/9226 [03:03<00:00, 50.17it/s]


In [31]:
def distance(a, b):
  numerator = a@b
  denominator = np.linalg.norm(a) * np.linalg.norm(b)
  return numerator/denominator

In [56]:
def distanceL1(a, b):
  return np.sum(np.absolute(a - b))
def distanceL2(a, b):
  return np.sqrt(np.sum((a-b)**2))

In [32]:
def vectorize(text, vocab):
  vec = []
  normalize_text = Normalize(text)
  for word in vocab:
    vec.append(normalize_text.count(word))
  return np.array(vec)

In [33]:
term_docs_matrix = {}
for (title, artical) in tqdm(docs_list):
  vec = vectorize(artical, vocab)
  term_docs_matrix[(title, artical)] = vec

100%|██████████| 9223/9223 [16:32<00:00,  9.29it/s]


In [49]:
def Ranking(query, term_docs_matrix, top):
  vec_query = vectorize(query, vocab)
  ranking = []
  for doc_info, vec in tqdm(term_docs_matrix.items()):
    score = distance(vec_query, vec)
    ranking.append((score, doc_info[0]))
    ranking.sort(reverse=True)
  for rank in ranking[:top]:
    print(rank)
  return ranking

In [50]:
query = 'Học sinh trung học'
rankings = Ranking(query,term_docs_matrix, 10)

100%|██████████| 9223/9223 [00:10<00:00, 884.06it/s]

(0.9703659148709143, 'Hà Nội không thu bất cứ khoản nào khi học sinh lớp 10 nhập học')
(0.9684335824677827, 'Bối rối chọn tổ hợp lớp 10: Chọn nhầm có được đổi lại hay không?')
(0.967774010163236, 'Phụ huynh, học sinh bất ngờ khi bỏ quy định trao giấy khen cho học sinh tiên tiến')
(0.9665850277785668, 'Thí sinh ở Trung Quốc bị trừ điểm thi vào lớp 10 nếu tăng độ cận thị')
(0.9665532877758299, 'Bộ GD&ĐT đề xuất miễn học phí cho học sinh THCS cả nước')
(0.9664766401967334, 'Thừa Thiên - Huế: Vinh danh học sinh đoạt huy chương Olympic')
(0.9662913772619467, 'Những lưu ý đối với học sinh trúng tuyển bổ sung vào lớp 10 của Hà Nội')
(0.9660630906728181, 'Chọn tổ hợp môn học lớp 10: Giải đáp những câu hỏi nóng - Tuổi Trẻ Online')
(0.9659502813911685, 'Lần đầu tiên có học sinh lớp 10 đoạt huy chương vàng Olympic Vật lý quốc tế')
(0.9659227169778635, 'Đề xuất miễn học phí cho học sinh THCS cả nước')


In [61]:
def RankingL1(query, term_docs_matrix, top):
  vec_query = vectorize(query, vocab)
  ranking = []
  for doc_info, vec in tqdm(term_docs_matrix.items()):
    score = distanceL1(vec_query, vec)
    ranking.append((score, doc_info[0]))
    ranking.sort(reverse=True)
  for rank in ranking[:top]:
    print(rank)
  return ranking

In [62]:
query = 'Học sinh trung học'
rankings = RankingL1(query,term_docs_matrix, 10)

100%|██████████| 9223/9223 [00:05<00:00, 1704.46it/s]

(46724, "Ủy ban Kiểm tra T.Ư chỉ ra cách Việt Á 'thâu tóm' kit xét nghiệm Covid")
(43730, 'Tập Yoga tại nhà với bài tập đơn giản cho người mới bắt đầu')
(40851, 'Thủ tướng: Lực lượng Cảnh sát nhân dân xứng đáng là đội quân Anh hùng | Báo Dân trí')
(39899, 'Kết luận của Bộ Chính trị về bảo đảm an ninh nguồn nước và an toàn đập, hồ chứa nước đến năm 2030, tầm nhìn đến năm 2045')
(37830, 'Làm phim tiểu sử và nỗi lo "Ở nhà Google cho nhanh, mà nó còn miễn phí"')
(36764, 'Triết lý của bầu Hiển & bí mật bên trong lò đào tạo giúp bóng đá Việt Nam vươn tầm châu Á')
(36282, 'Nhà sáng lập sàn TMĐT nông sản FoodMap: Từ trái hồng treo gió đến bản đồ nông - đặc sản Việt đồng hành với nông dân ba miền')
(36238, 'Tóc Tiên: Hoàng Touliver chỉ làm nhạc anh ấy thích, nhiều lần từ chối làm cho tôi')
(35879, '"Ngộp thở" trong chính nhà mình, nữ TS Việt ở Mỹ tìm ra lối sống giúp thành công vượt bậc')
(32333, 'Diễn văn của Tổng Bí thư Nguyễn Phú Trọng tại lễ kỷ niệm quan hệ Việt - Lào')


In [63]:
def RankingL2(query, term_docs_matrix, top):
  vec_query = vectorize(query, vocab)
  ranking = []
  for doc_info, vec in tqdm(term_docs_matrix.items()):
    score = distanceL2(vec_query, vec)
    ranking.append((score, doc_info[0]))
    ranking.sort(reverse=True)
  for rank in ranking[:top]:
    print(rank)
  return ranking

In [64]:
query = 'Học sinh trung học'
rankings = RankingL2(query,term_docs_matrix, 10)

100%|██████████| 9223/9223 [00:05<00:00, 1779.66it/s]

(19420.379501956188, "Ủy ban Kiểm tra T.Ư chỉ ra cách Việt Á 'thâu tóm' kit xét nghiệm Covid")
(18617.43226119005, 'Tập Yoga tại nhà với bài tập đơn giản cho người mới bắt đầu')
(17184.96310732147, 'Thủ tướng: Lực lượng Cảnh sát nhân dân xứng đáng là đội quân Anh hùng | Báo Dân trí')
(16619.421439990023, 'Kết luận của Bộ Chính trị về bảo đảm an ninh nguồn nước và an toàn đập, hồ chứa nước đến năm 2030, tầm nhìn đến năm 2045')
(15714.473010572134, 'Làm phim tiểu sử và nỗi lo "Ở nhà Google cho nhanh, mà nó còn miễn phí"')
(15563.084527175197, 'Triết lý của bầu Hiển & bí mật bên trong lò đào tạo giúp bóng đá Việt Nam vươn tầm châu Á')
(15156.593218794254, 'Tóc Tiên: Hoàng Touliver chỉ làm nhạc anh ấy thích, nhiều lần từ chối làm cho tôi')
(15055.664648231243, 'Nhà sáng lập sàn TMĐT nông sản FoodMap: Từ trái hồng treo gió đến bản đồ nông - đặc sản Việt đồng hành với nông dân ba miền')
(14925.02807367544, '"Ngộp thở" trong chính nhà mình, nữ TS Việt ở Mỹ tìm ra lối sống giúp thành công vượt